In [11]:
import pandas as pd
from glob import glob
from random import choice, sample
import altair as alt
from os.path import exists


In [12]:
country_filepaths = glob("*/*/*/*_data.csv")
print(country_filepaths)

['data/world_bank/poland/poland_data.csv', 'data/world_bank/belarus/belarus_data.csv', 'data/world_bank/russia/russia_data.csv', 'data/world_bank/norway/norway_data.csv', 'data/world_bank/luxembourg/luxembourg_data.csv', 'data/world_bank/cyprus/cyprus_data.csv', 'data/world_bank/ukraine/ukraine_data.csv', 'data/world_bank/france/france_data.csv', 'data/world_bank/moldova/moldova_data.csv', 'data/world_bank/san_marino/san_marino_data.csv', 'data/world_bank/georgia/georgia_data.csv', 'data/world_bank/bosnia_and_herzegovina/bosnia_and_herzegovina_data.csv', 'data/world_bank/belgium/belgium_data.csv', 'data/world_bank/netherlands/netherlands_data.csv', 'data/world_bank/morocco/morocco_data.csv', 'data/world_bank/estonia/estonia_data.csv', 'data/world_bank/spain/spain_data.csv', 'data/world_bank/montenegro/montenegro_data.csv', 'data/world_bank/serbia/serbia_data.csv', 'data/world_bank/croatia/croatia_data.csv', 'data/world_bank/australia/australia_data.csv', 'data/world_bank/azerbaijan/aze

## Testing With One Country

In [13]:
poland = pd.read_csv(country_filepaths[0])

In [14]:
poland.sample(10)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,Unnamed: 68
833,Poland,POL,Coverage of unemployment benefits and ALMP in ...,per_lm_alllm.cov_q4_tot,NaN,NaN,NaN,NaN,NaN,NaN,...,1.240984e+00,1.174196e+00,1.232780e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1036,Poland,POL,Average precipitation in depth (mm per year),AG.LND.PRCP.MM,NaN,600.0,600.0,600.0,600.0,600.0,...,6.000000e+02,6.000000e+02,6.000000e+02,6.000000e+02,6.000000e+02,6.000000e+02,NaN,NaN,NaN,NaN
377,Poland,POL,Short-term debt (% of total external debt),DT.DOD.DSTC.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1033,Poland,POL,"Communications, computer, etc. (% of service i...",BM.GSR.CMCP.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,5.151061e+01,5.131084e+01,5.164888e+01,5.218625e+01,5.310862e+01,6.005099e+01,5.749762e+01,55.211421,58.550622,NaN
1005,Poland,POL,"Stocks traded, total value (% of GDP)",CM.MKT.TRAD.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,1.103050e+01,9.710038e+00,1.296160e+01,9.298580e+00,8.454603e+00,1.391931e+01,1.151952e+01,9.514066,NaN,NaN
728,Poland,POL,"Share of tariff lines with specific rates, pri...",TM.TAX.TCOM.SR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,3.011401e+00,2.134678e+00,2.990922e+00,3.064476e+00,3.085244e+00,2.665311e+01,2.283862e+00,NaN,NaN,NaN
469,Poland,POL,Benefit incidence of unemployment benefits and...,per_lm_alllm.ben_q1_tot,NaN,NaN,NaN,NaN,NaN,NaN,...,2.577570e+01,2.345273e+01,2.050409e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156,Poland,POL,"Over-age students, primary (% of enrollment)",SE.PRM.OENR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,9.414200e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
756,Poland,POL,Women who were first married by age 18 (% of w...,SP.M18.2024.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
854,Poland,POL,"Adjusted net savings, including particulate em...",NY.ADJ.SVNG.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,4.555354e+10,4.176135e+10,4.575235e+10,5.511696e+10,6.183940e+10,6.380746e+10,7.014265e+10,NaN,NaN,NaN


In [15]:
poland.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 69 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Country Name    1492 non-null   object 
 1   Country Code    1492 non-null   object 
 2   Indicator Name  1492 non-null   object 
 3   Indicator Code  1492 non-null   object 
 4   1960            144 non-null    float64
 5   1961            158 non-null    float64
 6   1962            158 non-null    float64
 7   1963            163 non-null    float64
 8   1964            159 non-null    float64
 9   1965            162 non-null    float64
 10  1966            159 non-null    float64
 11  1967            159 non-null    float64
 12  1968            159 non-null    float64
 13  1969            160 non-null    float64
 14  1970            226 non-null    float64
 15  1971            227 non-null    float64
 16  1972            239 non-null    float64
 17  1973            236 non-null    f

In [16]:
poland_long = poland.melt(
        id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'],
        var_name='Year', value_name='Value'
    )

In [17]:
poland_long[poland_long["Value"].notna()].head().max()

Country Name                                                 Poland
Country Code                                                    POL
Indicator Name    Ratio of female to male labor force participat...
Indicator Code                                          SM.POP.TOTL
Year                                                           1960
Value                                                     2424881.0
dtype: object

In [18]:
pivoted_df = poland_long.pivot_table(
    index=['Country Name', 'Country Code', 'Year'],
    columns='Indicator Name',
    values='Value'
)

## Combining All Countries

Data is shaped as follows:
> [Country, Country Code, Year, [All Indicators]]

In [19]:
df_list = []

In [20]:
for country in country_filepaths:
    df = pd.read_csv(country)
    df["Indicator Name"].replace(":", "", inplace=True)
    df_long = df.melt(
        id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'],
        var_name='Year', value_name='Value'
    )
    pivoted_df = df_long.pivot_table(
    index=['Country Name', 'Country Code', 'Year'],
    columns='Indicator Name',
    values='Value'
    ).reset_index()
    df_list.append(pivoted_df)

/var/folders/kl/zs51p_z135905dzchw5f3n480000gp/T/ipykernel_16648/3594722023.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Indicator Name"].replace(":", "", inplace=True)
/var/folders/kl/zs51p_z135905dzchw5f3n480000gp/T/ipykernel_16648/3594722023.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values a

In [21]:
combined_data = pd.concat(df_list)

In [22]:
combined_data["Country Name"].unique()

array(['Poland', 'Belarus', 'Russian Federation', 'Norway', 'Luxembourg',
       'Cyprus', 'Ukraine', 'France', 'Moldova', 'San Marino', 'Georgia',
       'Bosnia and Herzegovina', 'Belgium', 'Netherlands', 'Morocco',
       'Estonia', 'Spain', 'Montenegro', 'Serbia', 'Croatia', 'Australia',
       'Azerbaijan', 'Latvia', 'Germany', 'Albania', 'Portugal',
       'Ireland', 'Switzerland', 'Lithuania', 'Sweden', 'United Kingdom',
       'Slovak Republic', 'Slovenia', 'Hungary', 'Monaco', 'Bulgaria',
       'Romania', 'Italy', 'Denmark', 'North Macedonia', 'Andorra',
       'Austria', 'Finland', 'Malta', 'Greece', 'Armenia', 'Czechia',
       'Israel', 'Turkiye', 'Iceland'], dtype=object)

In [23]:
combined_data.dropna(subset="Year", inplace=True)
combined_data = combined_data.astype({"Year": int})

In [24]:
for col in combined_data.columns:
    if combined_data[col].dtype != float:
        print(col, combined_data[col].dtype)

Country Name object
Country Code object
Year int64


In [25]:
cols = ['Country Name', 'Country Code', 'Year'] + [col for col in combined_data if col not in ['Country Name', 'Country Code', 'Year', "Indicator Name"]]
combined_data = combined_data[cols].reset_index()

In [26]:
combined_data.sample(10)

Indicator Name,index,Country Name,Country Code,Year,Access to clean fuels and technologies for cooking (% of population),"Access to clean fuels and technologies for cooking, rural (% of rural population)","Access to clean fuels and technologies for cooking, urban (% of urban population)",Access to electricity (% of population),"Access to electricity, rural (% of rural population)","Access to electricity, urban (% of urban population)",...,"People with basic handwashing facilities including soap and water, urban (% of urban population)","Risk premium on lending (lending rate minus treasury bill rate, %)","Incidence of malaria (per 1,000 population at risk)","Net financial flows, RDB concessional (NFL, current US$)",Financial intermediary services indirectly Measured (FISIM) (constant LCU),"Net financial flows, IMF concessional (NFL, current US$)",Newborns protected against tetanus (%),"Net official flows from UN agencies, UNWTO (current US$)",Children with fever receiving antimalarial drugs (% of children under age 5 with fever),Use of insecticide-treated bed nets (% of under-5 population)
2274,34,Bulgaria,BGR,1994,NaN,NaN,NaN,100.000000,100.000000,100.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2091,43,Slovenia,SVN,2003,100.0,100.0,100.0,100.000000,100.000000,100.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,12,Belarus,BLR,1972,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1314,34,Australia,AUS,1994,NaN,NaN,NaN,100.000000,100.000000,100.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019,35,Slovak Republic,SVK,1995,NaN,NaN,NaN,100.000000,100.000000,100.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1368,24,Azerbaijan,AZE,1984,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
929,33,Morocco,MAR,1993,NaN,NaN,NaN,56.772652,22.645689,90.131622,...,NaN,NaN,NaN,1084997.0,NaN,NaN,84.0,NaN,NaN,NaN
2855,39,Greece,GRC,1999,NaN,NaN,NaN,100.000000,100.000000,100.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2123,11,Hungary,HUN,1971,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1272,56,Croatia,HRV,2016,100.0,100.0,100.0,100.000000,100.000000,100.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
combined_data.columns = combined_data.columns.str.replace(':', '', regex=False)

## Choosing a random Indicator to Check For 3 Random Countries

In [40]:
randomly_chosen_countries = sample(combined_data['Country Name'].unique().tolist(), 7)
filtered_data = combined_data[combined_data['Country Name'].isin(randomly_chosen_countries)]
randomly_chosen_countries


['Turkiye',
 'Spain',
 'Switzerland',
 'North Macedonia',
 'Latvia',
 'Malta',
 'Hungary']

In [41]:
randomly_chosen_column = choice([col for col in combined_data if col not in ['Country Name', 'Country Code', 'Year', "Indicator Name"]])
randomly_chosen_column 

'Market capitalization of listed domestic companies (current US$)'

In [42]:
chart = alt.Chart(filtered_data).mark_line().encode(
    x='Year:O',
    y=alt.Y(randomly_chosen_column.replace(":", r"\:"), title=f'{randomly_chosen_column}'),
    color="Country Name"
).properties(
    title=f'{randomly_chosen_column} by Year'
)
chart.show()

In [31]:
combined_data.to_csv("./data/world_bank/combined_world_data.csv", index=False)